In [17]:
import os
import sys
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../src"))

In [18]:
# Import functions from the model_training script
from model_training import preprocess_target_column, split_data, train_logistic_regression, train_random_forest, evaluate_model
import pandas as pd

In [19]:
#Load the processed data
processed_data_path = "D:/Kifya_training/Week 6/Bati-Bank-Credit-Scoring-Model/data/processed/processed_data.csv"
df = pd.read_csv(processed_data_path)

In [20]:
# Preprocess the target column
df = preprocess_target_column(df, "FraudResult")

In [21]:
print("Unique values in target column (FraudResult) after preprocessing:")
print(df["FraudResult"].unique())
print("Unique values in target column (FraudResult) after preprocessing:")
print(df["FraudResult"].unique())


Unique values in target column (FraudResult) after preprocessing:
[0 1]
Unique values in target column (FraudResult) after preprocessing:
[0 1]


In [22]:
# Define the target column and split the data
target_column = "FraudResult"
X_train, X_test, y_train, y_test = split_data(df, target_column)

In [23]:
# Train Logistic Regression
log_model = train_logistic_regression(X_train, y_train)
print("Logistic Regression Model Trained.")

Logistic Regression Model Trained.


In [24]:
# Train Random Forest
rf_model = train_random_forest(X_train, y_train)
print("Random Forest Model Trained.")


Random Forest Model Trained.


In [25]:
# Evaluate Logistic Regression
log_metrics = evaluate_model(log_model, X_test, y_test)
print("Logistic Regression Metrics:", log_metrics)

Logistic Regression Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'roc_auc': np.float64(1.0)}


In [26]:
# Evaluate Random Forest
rf_metrics = evaluate_model(rf_model, X_test, y_test)
print("Random Forest Metrics:", rf_metrics)

Random Forest Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'roc_auc': np.float64(1.0)}


In [27]:
# Evaluate Logistic Regression
log_metrics = evaluate_model(log_model, X_test, y_test)
print("Logistic Regression Metrics:")
for metric, value in log_metrics.items():
    print(f"{metric}: {value:.4f}")

# Evaluate Random Forest
rf_metrics = evaluate_model(rf_model, X_test, y_test)
print("\nRandom Forest Metrics:")
for metric, value in rf_metrics.items():
    print(f"{metric}: {value:.4f}")


Logistic Regression Metrics:
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000
roc_auc: 1.0000

Random Forest Metrics:
accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000
roc_auc: 1.0000


In [28]:
print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])


Training set size: 76529
Test set size: 19133


In [29]:
print("Target distribution in training set:")
print(y_train.value_counts(normalize=True))
print("\nTarget distribution in test set:")
print(y_test.value_counts(normalize=True))


Target distribution in training set:
FraudResult
0    0.997948
1    0.002052
Name: proportion, dtype: float64

Target distribution in test set:
FraudResult
0    0.998118
1    0.001882
Name: proportion, dtype: float64


In [30]:
# Ensure no overlap in indices
train_indices = set(X_train.index)
test_indices = set(X_test.index)

print("Overlap in train-test indices:", len(train_indices.intersection(test_indices)))


Overlap in train-test indices: 0


In [31]:
# Compute correlation between features and the target variable
correlation = df.corr()["FraudResult"].sort_values(ascending=False)
print("Correlation of features with target variable:")
print(correlation)


ValueError: could not convert string to float: 'TransactionId_76871'